# Import data

In [2]:
import pandas as pd
from sklearn.impute import SimpleImputer
import numpy as np

data = pd.read_csv('ml-03-data-processing-songs-dataset.csv')

# Drop missing values

In [3]:
# Drop columns
for x,y in enumerate(data.isna().sum()): #identify missing values in each column
    if y > data.shape[0]/2: #if missing value > 50% of rows
        data.drop(data.columns[x],axis=1,inplace=True)
        print(f'Successfully dropped column: {data.columns[x]}, with {y} null values')

# Drop rows
Index_rows_missing = data[data.isna().sum(axis=1)>= data.shape[1]/2].index # index of rows with missing values > 50%
if Index_rows_missing.size > 0:
    print(f'Successfully dropped {len(Index_rows_missing)} rows.')
    data.drop(Index_rows_missing,axis=0,inplace=True)

Successfully dropped column: Loudness (dB), with 1024 null values
Successfully dropped column: Acousticness, with 1034 null values
Successfully dropped 8 rows.


# Impute Missing Values

In [4]:
#isolate columns by datatype
columns_numeric = data.select_dtypes(include=np.number)
columns_categorical = data.select_dtypes(include=['object'])

#fit using sklearn
imputer = SimpleImputer(strategy='mean').fit_transform(columns_numeric)
transformed_numeric = pd.DataFrame(imputer) #convert transformed data into pd.dataframe type

imputer = SimpleImputer(strategy='most_frequent').fit_transform(columns_categorical)
transformed_categorical = pd.DataFrame(imputer)

#rename column names
transformed_numeric.columns = columns_numeric.columns
transformed_categorical.columns = columns_categorical.columns

#reassemble into dataframe
data_new = pd.concat([transformed_numeric,transformed_categorical],axis=1)


# Identifying outliers

In [5]:
#Identifying outliers 3 standard deviations away
means = data.describe().loc['mean']
bounds = data.describe().loc['std']*3
columns = data.describe().columns
outliers = []
for i in range(len(means)):
    upperbound = means[i] + bounds[i]
    lowerbound = means[i] - bounds[i]
    outliers = list(filter(lambda x: x > upperbound or x < lowerbound,data_new[columns[i]]))
    outliers.sort()
    if len(outliers) > 0: # if outlier exists
        print(f'{columns[i]} - (Outliers: {len(outliers)}|{round(len(outliers)/len(data_new)*100,2)}%)')
        print(f'\tList: {outliers}')
        print()

Year - (Outliers: 35|1.76%)
	List: [92.0, 92.0, 92.0, 92.0, 92.0, 92.0, 92.0, 92.0, 92.0, 92.0, 92.0, 92.0, 92.0, 92.0, 92.0, 92.0, 92.0, 92.0, 92.0, 92.0, 92.0, 92.0, 92.0, 92.0, 92.0, 92.0, 92.0, 92.0, 92.0, 92.0, 92.0, 92.0, 92.0, 92.0, 92.0]

Beats Per Minute (BPM) - (Outliers: 4|0.2%)
	List: [15.0, 18.0, 297.0, 305.0]

Loudness (dB) - (Outliers: 22|1.11%)
	List: [-27.0, -24.0, -24.0, -22.0, -22.0, -22.0, -22.0, -22.0, -22.0, -21.0, -21.0, -21.0, -21.0, -21.0, -21.0, -21.0, -21.0, -20.0, -20.0, -20.0, -20.0, -20.0]

Liveness - (Outliers: 57|2.87%)
	List: [70.0, 70.0, 70.0, 71.0, 71.0, 72.0, 72.0, 73.0, 73.0, 73.0, 73.0, 76.0, 76.0, 76.0, 76.0, 76.0, 77.0, 77.0, 78.0, 78.0, 79.0, 80.0, 81.0, 81.0, 82.0, 83.0, 83.0, 84.0, 85.0, 85.0, 85.0, 85.0, 86.0, 87.0, 87.0, 89.0, 90.0, 91.0, 91.0, 92.0, 92.0, 93.0, 93.0, 94.0, 95.0, 95.0, 96.0, 96.0, 96.0, 97.0, 97.0, 97.0, 97.0, 97.0, 98.0, 99.0, 99.0]

Speechiness - (Outliers: 42|2.11%)
	List: [19.0, 19.0, 19.0, 19.0, 19.0, 20.0, 20.0, 20.0, 

In [ ]:
data_new[columns[0]].sort_values(ascending=False)

# Date modification

In [6]:
from datetime import datetime,date
# Convert Month (String) to Numeric
try:
    data_new['Month'] = data_new['Month'].apply(lambda x: str(datetime.strptime(x,'%b').month) if not x.isnumeric() else x)
except:
    print('Error when converting')
    
# Handle incorrect values in Year column
for index in data_new[data_new.Year < 1000].Year.index:
    data_new['Year'].at[index] = 1992

data_new['Year'] = data_new['Year'].astype(int) # convert to int, remove decimals

# Convert Date to Datetime (new column)

In [7]:
age = []
data_new['day']= '01' #create a day column for passing value.
_date = pd.concat([data_new['Year'],data_new['Month'],data_new['day']],axis=1)

column_date = pd.to_datetime(_date,format='%m/%Y/%d') #convert to date format
for x in column_date.iteritems():
    age.append(date.today().year - x[1].year)

data_new['Age'] = age # Assign new column: age


In [8]:
# Delete other date format columns
data_new.drop(['Year','Month','day'],axis=1,inplace=True)
data_new

,Index,Beats Per Minute (BPM),Energy,Loudness (dB),Liveness,Valence,Acousticness,Speechiness,Popularity,Title,Artist,Top Genre,Age
0,1.0,157.0,30.0,-14.0,11.0,68.0,94.0,3.0,71.0,Sunrise,Norah Jones,adult standards,18
1,2.0,135.0,79.0,-11.0,17.0,81.0,17.0,7.0,39.0,Black Night,Deep Purple,album rock,22
2,3.0,168.0,69.0,-9.0,7.0,52.0,2.0,17.0,69.0,Clint Eastwood,Gorillaz,alternative hip hop,21
3,4.0,173.0,96.0,-4.0,3.0,37.0,0.0,4.0,76.0,The Pretender,Foo Fighters,alternative metal,15
4,5.0,106.0,82.0,-5.0,10.0,87.0,1.0,3.0,59.0,Waitin' On A Sunny Day,Bruce Springsteen,classic rock,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1981,1990.0,94.0,21.0,-12.0,11.0,72.0,84.0,7.0,63.0,Heartbreak Hotel,Elvis Presley,adult standards,64
1982,1991.0,175.0,76.0,-8.0,76.0,95.0,73.0,6.0,69.0,Hound Dog,Elvis Presley,adult standards,64
1983,1992.0,168.0,80.0,-9.0,31.0,97.0,74.0,7.0,74.0,Johnny B. Goode,Chuck Berry,blues rock,63
1984,1993.0,174.0,26.0,-13.0,7.0,60.0,54.0,4.0,65.0,Take Five,The Dave Brubeck Quartet,bebop,63


# Create Dummy variables for categorical variables

In [ ]:
pd.get_dummies(data_new.select_dtypes(include='object'),drop_first=True)

In [24]:
y = data_new['Popularity']
X = data_new.drop('Popularity',axis=1)

In [15]:
# create split
import sklearn.model_selection
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y,test_size=0.3,train_size=0.7,random_state=12)

In [22]:
X_train.describe()

,Index,Beats Per Minute (BPM),Energy,Loudness (dB),Liveness,Valence,Acousticness,Speechiness,Age
count,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000
mean,992.865468,119.772449,59.853929,-8.948221,19.176263,50.617620,28.372551,5.044604,28.946763
std,575.441340,28.744915,22.107251,3.652496,16.611337,16.989649,28.853229,4.529818,16.122016
min,1.000000,18.000000,4.000000,-27.000000,2.000000,6.000000,0.000000,2.000000,3.000000
25%,501.250000,98.000000,43.000000,-11.000000,9.000000,50.000000,3.000000,3.000000,14.250000
50%,999.000000,119.000000,62.000000,-8.000000,13.000000,50.820833,18.000000,4.000000,28.000000
75%,1484.500000,136.000000,78.000000,-6.000000,23.000000,50.820833,49.000000,5.000000,43.000000
max,1994.000000,305.000000,100.000000,-2.000000,97.000000,98.000000,99.000000,46.000000,66.000000


In [23]:
X_test.describe()

,Index,Beats Per Minute (BPM),Energy,Loudness (dB),Liveness,Valence,Acousticness,Speechiness,Age
count,596.000000,596.000000,596.000000,596.000000,596.000000,596.000000,596.000000,596.000000,596.000000
mean,1004.755034,121.312081,59.186242,-9.166130,18.595641,51.294771,30.206246,4.870793,29.283557
std,578.082001,28.315389,22.273107,3.643932,16.975590,18.154407,29.366356,4.106779,16.196020
min,4.000000,15.000000,3.000000,-22.000000,3.000000,5.000000,0.000000,2.000000,3.000000
25%,475.750000,102.000000,41.750000,-11.000000,9.000000,50.365625,4.000000,3.000000,15.000000
50%,995.500000,119.000000,60.000000,-9.000000,12.000000,50.820833,19.000000,4.000000,29.000000
75%,1505.250000,136.000000,78.000000,-6.000000,21.000000,52.000000,54.000000,5.000000,43.000000
max,1992.000000,297.000000,99.000000,-2.000000,99.000000,98.000000,98.000000,55.000000,64.000000
